# Exp 4. Simple modeling with almost no pp/fe 

In [1]:
import os
import sys
from pathlib import Path

PROJECT_DIR = os.getcwd() + "/../../"
sys.path.insert(0, PROJECT_DIR)

In [2]:
# step 1. Load global variables
from src.exp.exp4_simple.step1_global_variables import *

2019-10-31 06:23:48,034 - INFO - logger set up
2019-10-31 06:23:48,036 - INFO - seed=42
